In [3]:
from bertopic import BERTopic
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)
from umap import UMAP
import sys
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from openpyxl import Workbook

import nltk
from nltk.corpus import stopwords

from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer
from nltk.tokenize import sent_tokenize, word_tokenize

/Users/a788/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/a788/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
link = 'documents.xlsx'
df = pd.ExcelFile(link).parse('general_labeled')
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)
df_ns = df['narratives_self']

# drop any missing or na values from the dataframe
df_ns = df_ns.dropna()
df_ns = df_ns.astype(str)

In [5]:
len(df['ns_label_unmerged'].unique())

20

In [6]:
stop_words = stopwords.words('english')

free_s_w = ["shirt", "shirts", "would", "tshirt", "could", "decision", "decide", "decisions", "decided"] 
stop_words.extend(free_s_w)

In [7]:
!pip install -U spacy
!python3 -m spacy download en_core_web_sm
!pip install hf_xet

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
/Users/a788/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━

In [ ]:
import spacy
spacy.load('en_core_web_sm')
keybert_model = KeyBERTInspired()

pos_model = PartOfSpeech("en_core_web_sm")

mmr_model = MaximalMarginalRelevance(diversity=0.4)

representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "POS": pos_model
}

In [ ]:
from sklearn.decomposition import PCA
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df_ns, show_progress_bar=True)

# we test with dimensionality reduction with UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=40)
umap_model = UMAP(random_state=40)

hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=1, metric='euclidean', cluster_selection_method='leaf' ,prediction_data=True)
vectorizer_model = CountVectorizer(stop_words=stop_words, min_df=1, ngram_range=(1,1))

ctfidf_model = ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

Batches: 100%|██████████| 25/25 [00:10<00:00,  2.31it/s]


In [ ]:
BERT_model = BERTopic(
    embedding_model=embedding_model,          
    umap_model=umap_model,                    
    hdbscan_model=hdbscan_model,              
    vectorizer_model=vectorizer_model,        
    ctfidf_model=ctfidf_model, 
    representation_model=representation_model
)

In [11]:
topics, _ = BERT_model.fit_transform(df_ns)
topics_no_outliers = BERT_model.reduce_outliers(df_ns, topics)

In [12]:
top_inf = BERT_model.get_topic_info()

## Can you please explain your reasoning/thought process when deciding how much you were willing to pay sellers in different scenarios?

In [13]:
top_inf.to_excel('final_ns_topics.xlsx')

In [14]:
top_inf

Topic  Count                                    Name  \
0      -1    120           -1_privacy_donate_cause_study   
1       0     89         0_climate_change_justice_racial   
2       1     79       1_company_support_business_causes   
3       2     75           2_bonus_payment_rather_prefer   
4       3     70       3_service_product_sellers_quality   
5       4     57         4_based_thought_considered_much   
6       5     53          5_lgbtq_rights_equality_gender   
7       6     42                 6_plain_high_15_clothes   
8       7     39    7_charity_donation_minimum_typically   
9       8     38  8_bonus_sellerâ_confidence_opportunity   
10      9     34           9_gun_sales_opportunity_still   
11     10     34         10_views_company_politics_cares   
12     11     25              11_need_money_instead_dont   
13     12     23        12_guessed_asked_guess_basically   
14     13     22            13_exploit_try_profit_stance   

                                                                                 Representation  \
0              [privacy, donate, cause, study, right, seller, iâ, genuine, really, information]   
1         [climate, change, justice, racial, faith, traditional, fighting, people, think, feel]   
2    [company, support, business, causes, product, cause, someone, political, views, something]   
3           [bonus, payment, rather, prefer, cash, personal, desire, currently, need, compared]   
4   [service, product, sellers, quality, offers, items, reputation, prices, deciding, personal]   
5              [based, thought, considered, much, quality, well, mainly, retail, guided, first]   
6                    [lgbtq, rights, equality, gender, gun, gay, two, guns, advocates, support]   
7                   [plain, high, 15, clothes, 10, spend, usually, material, amount, necessity]   
8                  [charity, donation, minimum, typically, blank, donate, hold, tee, plain, 20]   
9    [bonus, sellerâ, confidence, opportunity, accept, reward, seller, choice, online, instead]   
10           [gun, sales, opportunity, still, benefit, fight, rights, donating, portion, might]   
11      [views, company, politics, cares, additionally, care, think, values, political, issues]   
12                             [need, money, instead, dont, offered, rather, iâ, cash, tis, id]   
13        [guessed, asked, guess, basically, lie, shoes, respond, unbiasely, guesses, supposed]   
14    [exploit, try, profit, stance, boost, human, understanding, potentially, guess, relevant]   

                                                                                                       KeyBERT  \
0                      [donation, sellers, sales, money, charitable, seller, selling, purchase, value, donate]   
1            [climate, capitalism, beliefs, endorse, sincerity, belief, values, profits, society, initiatives]   
2            [endorses, charities, donate, values, ideology, sellers, money, supporting, political, purchases]   
3                       [monetary, value, price, worth, money, considerations, amount, cost, payment, vendors]   
4    [value, sellers, consideration, prices, evaluating, considerations, seller, negotiation, items, deciding]   
5                               [value, thrift, tshirts, worth, costs, price, sellers, discount, cost, demand]   
6                      [equality, advocates, values, sellers, lgbtq, endorse, lgbt, preference, rights, sales]   
7                        [thrift, clothes, inexpensive, cheap, goodwill, spend, price, cost, buy, fashionable]   
8                           [donation, donate, charity, donating, tee, value, worth, dollars, price, clothing]   
9                    [purchase, money, seller, valuation, value, reward, prizes, values, payment, preferences]   
10                        [sellers, sales, donate, seller, interest, donating, selling, sell, amount, tshirts]   
11                        [worth, values, buy, money, purchase, products, retailers, compa

In [ ]:
fig = BERT_model.visualize_documents(df_ns, embeddings=embeddings)

fig.update_traces(marker_showscale=False)

fig.update_layout(showlegend=False)

fig.show()

In [16]:
number_of_topics = len(BERT_model.get_topic_info())
fig = BERT_model.visualize_barchart(top_n_topics=number_of_topics)
fig

In [17]:
fig2 = BERT_model.visualize_hierarchy()
fig2

In [18]:
%pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [19]:
docs = df_ns.tolist()

In [20]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})

In [22]:
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = BERT_model._preprocess_text(documents_per_topic.Document.values)

In [ ]:
vectorizer = BERT_model.vectorizer_model
analyzer = vectorizer_model.build_analyzer()

In [ ]:
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in BERT_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

In [25]:
len(topic_words)

14

Evaluation

In [ ]:
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
coherence_model.get_coherence_per_topic()

[0.5996331547331826,
 0.45680726184513515,
 0.5154689574578023,
 0.568718907312437,
 0.42553306700634386,
 0.6557107214238853,
 0.5307283391912401,
 0.5004640745302434,
 0.2928205149089979,
 0.5181215569662116,
 0.4739330951255261,
 0.5594901423865223,
 0.72470022324749,
 0.5670830526072543]

In [28]:
coherence_model.get_coherence()

0.5278009334815909